In [ ]:
!pip install chromadb==0.4.6
!pip install pydantic==1.10
# due to dependency issues i have used specific version for pydantic and chromadb libraries
!pip install sentence-transformers

In [ ]:
!pip install langchain
!pip install huggingface_hub

In [ ]:
!pip install transformers

In [ ]:
from langchain import HuggingFaceHub
from langchain.chains import ConversationChain

In [ ]:
#loading the API key
import getpass
import os
os.environ['HUGGING_FACE_HUB_API_KEY'] = getpass.getpass('Hugging face api key:')

Hugging face api key:··········


In [ ]:
# using this repo as human based interaction is most suitable with it, refference Jillani Soft Tech "Medium"
repo_id = 'lmsys/fastchat-t5-3b-v1.0'  # has 3B parameters: https://huggingface.co/lmsys/fastchat-t5-3b-v1.0
llm = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGING_FACE_HUB_API_KEY'],
                     repo_id=repo_id,
                     model_kwargs={'temperature':1e-10, 'max_length':32})

In [ ]:
import os
import getpass

from langchain.document_loaders import PyPDFLoader  #document loader: https://python.langchain.com/docs/modules/data_connection/document_loaders
from langchain.text_splitter import RecursiveCharacterTextSplitter  #document transformer: text splitter for chunking
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import PromptTemplate
from langchain.vectorstores import Chroma #vector store
from langchain import HuggingFaceHub  #model hub
from langchain.chains import RetrievalQA

from langchain.memory import ConversationBufferMemory

In [ ]:
!pip install pypdf

In [ ]:
path = input("Enter PDF file path: ")#"using the path as file uploaded to collab"
loader = PyPDFLoader(path)
pages = loader.load()

Enter PDF file path: /content/blockchain.pdf


In [ ]:
#number of pages
len(pages)

31

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
docs = splitter.split_documents(pages)

In [ ]:
embeddings = HuggingFaceEmbeddings()
doc_search = Chroma.from_documents(docs, embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query = "What is Blockchain"
similar_docs = doc_search.similarity_search(query, k=3)

In [ ]:
repo_id = 'lmsys/fastchat-t5-3b-v1.0'  # has 3B parameters: https://huggingface.co/lmsys/fastchat-t5-3b-v1.0
llm = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGING_FACE_HUB_API_KEY'],
                     repo_id=repo_id,
                     model_kwargs={'temperature':1e-10, 'max_length':32})

In [ ]:
template = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the question:
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

In [58]:
memory = ConversationBufferMemory(
    memory_key="history",
    input_key="question"
)

retrieval_chain = RetrievalQA.from_chain_type(llm,
                                              chain_type='stuff',
                                              retriever=doc_search.as_retriever(),
                                              chain_type_kwargs={
                                                  "prompt": prompt,
                                                  "memory": memory
                                              })